# Z0 Resonance

In [1]:
# Import the fundamental libraries
import numpy as np
import scipy
import sympy
import ROOT
from numpy.linalg import norm
from scipy.optimize import minimize
from IPython.display import IFrame

Welcome to JupyROOT 6.14/00


Define a function `read_events(file_name)` to read the events from the dat-files.

In [5]:
def read_events(file_name):
    """ Read and select event from the DAT-files. """
    
    # Prepareing the lists for information storage
    file_content = []
    
    with open(file_name) as f:
        
        # Loop over the dat-file
        for line in f.readlines():

            # If the line has only one entry, it indicates that the following lines describe
            # an event with this number of particles.
            # Such line is therefore not saved,
            # but it is used to create the structure of the event.
            if len(line.split( ))==1:

                # Initialize the lists for event storage (summary)
                event_content = []
                event_var = np.zeros(1, dtype = [('n_elec','int64'),('n_muon','int64'),('n_hadron','int64'),('n_total','int64'),
                                                 ('max_pt','float64'),('E_total','float64'),
                                                 ('pass_hadron_selection','int64'),('pass_muon_selection','int64')])
                #E_trans = 0
                #E_paral = 0

                # The total number of particles
                event_var['n_total'] = int(line.split( )[0])
                
                # Initialize the matrix for p-vector storage
                p_vec = np.zeros((event_var['n_total'][0],3))
                
                # Initialize the counter of particles
                # which indicates on which particle of this event we are now focusing
                count = 0

                pass


            # When there are more entries than a sigle number,
            # that's the parameters of a particle in an event.
            # The information about its momentum and mass are saved in a structured array.
            # another couple of variables are also added
            if len(line.split( ))!=1 and count<=event_var['n_total']:
                
                # Start with a new particle
                count += 1

                # Preparing the ndarray for line storage
                # (Alternative: define them as classes)
                line_content=np.zeros(1, dtype = [('px','float64'),('py', 'float64'),('pz', 'float64'),('m0', 'float64'),
                                                  ('pt', 'float64'),('p', 'float64'),('E', 'float64')])

                # Assign the 4 values of that line to px, py, pz and m
                for i, element in enumerate(line.split( )):
                    if i==1:
                        line_content['px'] = float(element)
                        p_vec[count-1,0] = line_content['px']
                    if i==2:
                        line_content['py'] = float(element)
                        p_vec[count-1,1] = line_content['py']
                    if i==3:
                        line_content['pz'] = float(element)
                        p_vec[count-1,2] = line_content['pz']
                    if i==4:
                        line_content['m0'] = abs(float(element))

                # Assign the remaining values of line_content
                line_content['pt'] = np.sqrt(line_content['px']**2 + line_content['py']**2)
                line_content['p'] = np.sqrt(line_content['px']**2 + line_content['py']**2 + line_content['pz']**2)
                line_content['E'] = np.sqrt(line_content['m0']**2 + line_content['p']**2)

                # Save (stack) the info of this particle
                # to a list of ndarrays
                event_content.append(line_content)

                # Calculate the parameters of the summary of the event using cumulation methode
                if abs(line_content['m0'][0]-0.000)<0.001: 
                    # Note that the type of line_content['m'] is 'numpy.ndarray' 
                    # while line_content['m'][0] is 'numpy.float64'
                    event_var['n_elec'] += 1    
                
                if abs(line_content['m0'][0]-0.106)<0.001:
                    event_var['n_muon'] += 1    
                
                if abs(line_content['m0'][0]-0.140)<0.001:
                    event_var['n_hadron'] += 1

                if line_content['pt']>event_var['max_pt']:
                    event_var['max_pt'] = line_content['pt']

                event_var['E_total'] += line_content['E']
                #E_trans = np.sqrt(line_content['m0']**2 + line_content['pt']**2)
                #E_paral = event_var['E_total'] - E_trans
                
                # When we arrive at the last particle (Notice that the counting is conducted at first)
                if count==event_var['n_total']:
                    
                    res = minimize(thrust, n0, args=p_vec, method='SLSQP', constraints=cons)
                    cos_th = cos_theta(res.x)
                    
                    # Executing the cuts
                    if ( event_var['E_total']/91>0.5 and event_var['E_total']/91<1.5
                            and ((event_var['n_total']>=13 and abs(cos_th)<0.74) or (event_var['n_total']>=17 and abs(cos_th)>0.74)) ):
                        event_var['pass_hadron_selection'] = 1
                    

                    if ( event_var['n_muon']==2 and event_var['n_total']<11
                            and event_var['max_pt']>25 and event_var['max_pt']<57
                            and event_var['E_total']>80 and event_var['E_total']<120 and abs(cos_th)<0.8 ):
                        event_var['pass_muon_selection'] = 1
                    
                    # Save the info of this event
                    # to a list of [[ndarray1,ndarray2...], ndarray]
                    file_content.append([event_content, event_var])
                
                pass


    return file_content

In [6]:
def thrust(thrust_axis, p):
    """ Calculate the opposite number of thrust: -T """
    numrtr = 0
    dnmntr = 0
    for i in np.arange(np.shape(p)[0]):
        numrtr += norm(p[i]*thrust_axis)
        dnmntr += norm(p[i])
    return -numrtr/dnmntr

def norm_n(n_vec):
    """ Normalization of thrust vector """
    return norm(n_vec) - 1

def cos_theta(n_vec):
    """ Calculate the cos value of polar angle """
    nx, ny, nz = n_vec
    return nz/norm(n_vec)

# The initial guess of the n-vector
n0 = np.asarray([0.2, 0.2, 0.2])
# Constrains of the n-vector
cons = {'type':'eq', 'fun':norm_n}

In [7]:
# Now let's move to the real files
# Here we declare the input and output files and we loop over the events
input_names = ['89gev.dat','91gev.dat','93gev.dat','hadrons.dat','muons.dat']
output_names = ['89gev.root','91gev.root','93gev.root','hadrons.root','muons.root']

# Particle variables and histograms limits
list_particle_var = ['px','py','pz','m0','pt','p','E']
list_particle_var = []
list_particle_var_limit = [20,20,20,1,20,20,20]

# Event variables and histogram limits
list_event_var = ['n_elec','n_muon','n_hadron','n_total','max_pt','E_total','pass_hadron_selection','pass_muon_selection']
list_event_var_limit = [20,20,100,100,100,200,2,2]

# Particle variables with additional conditions
list_particle_cut_names = ['is_muon']
list_particle_cut = ["abs(PARTICLE['m'][0]-0.106)<0.001"]

# Event variables with additional conditions
list_event_cut_names = ['hadron_cut','muon_cut']
list_event_cut = ["EVENT_VAR['pass_hadron_selection']>0.1","EVENT_VAR['pass_muon_selection']>0.1"]


# Loop over output-file names
for i, ofile in enumerate(output_names):
    
    # Create or overwrite the ROOT-file as output
    histo_ofile = ROOT.TFile(ofile,'RECREATE')
    
    # Declare the dict to storage the histograms and their names
    # since its contant can be aceessed by referring its name (compared with list)
    histo_particle = {}
    histo_event = {}
    
    # Creat empty histograms
    for j, var_j in enumerate(list_particle_var):
        # TH1F("histName", "histTitle", num_bins, x_low, x_high)
        histo_particle[var_j] = ROOT.TH1F(var_j, var_j, 200, -list_particle_var_limit[j], list_particle_var_limit[j])
        
        for k, var_k in enumerate(list_particle_cut_names):
            histo_particle[var_k+'_'+var_j] = ROOT.TH1F(var_k+'_'+var_j, var_k+'_'+var_j, 200, -list_particle_var_limit[j], list_particle_var_limit[j])
    
    for j, var_j in enumerate(list_event_var):
        histo_event[var_j] = ROOT.TH1F(var_j, var_j, 200, -list_event_var_limit[j], list_event_var_limit[j])
        
        for k, var_k in enumerate(list_event_cut_names):
            histo_event[var_k+'_'+var_j] = ROOT.TH1F(var_k+'_'+var_j, var_k+'_'+var_j, 200, -list_event_var_limit[j], list_event_var_limit[j])
    

    # Read events from input-files
    all_events = read_events(input_names[i])
    

    # Loop over all the events. Each element of all_events is a two-entry list [event_contant, event_var],
    # which includes a list of partiles and a ndarray of event variables.
    for EVENT, EVENT_VAR in all_events:
        
        # Fill the histograms. Each variable can be easily accessed by using its name. 
        # First we fill the histograms related to event variables.
        for j, var_j in enumerate(list_event_var):
            # Histogram for event variables without cuts
            histo_event[var_j].Fill(EVENT_VAR[var_j][0])
            
            # Histogram for event variables for hadronic events and muonic events ONLY!
            for k, var_k in enumerate(list_event_cut_names):
                if (eval(list_event_cut[k])):
                    histo_event[var_k+'_'+var_j].Fill(EVENT_VAR[var_j][0])
        
        # Next we fill the histograms with particle variables
        for PARTICLE in EVENT:
            for j, var_j in enumerate(list_particle_var):
                # Histogram of particle variables without cuts
                histo_particle[var_j].Fill(PARTICLE[var_j][0])
                
                # Histogram particle variables for muons ONLY!
                for k, var_k in enumerate(list_particle_cut_names):
                    if (eval(list_particle_cut[k])):
                        histo_particle[var_k+'_'+var_j].Fill(PARTICLE[var_j][0])
    
    # Write and save the histograms in ROOT-files
    histo_ofile.Write()
    histo_ofile.Close()

In [8]:
def compare_with_mc(data_name, var_name):
    """ Compare and plot the histograms or distributions respecting the variable (var_name)
    obtained from real data (data_name) and from Monte-Carlo data in the same graph. """

    # Create a new canvas for display
    # TCanvas("name", "title", win_width, win_height)
    canvas1 = ROOT.TCanvas(var_name, var_name)

    # We retrieve the histogram of the data to be compared
    ROOT_FILE_INPUT = ROOT.TFile(data_name, 'OPEN')
    HISTO_INPUT = ROOT_FILE_INPUT.Get(var_name)
    HISTO_INPUT.SetLineColor(4)

    # Next, retrieve the histograms from MC data for both hadron and muon
    ROOT_FILE_MC_HADRON = ROOT.TFile('hadrons.root', 'OPEN')
    HISTO_MC_HADRON = ROOT_FILE_MC_HADRON.Get(var_name)
    HISTO_MC_HADRON.SetLineColor(2)
    
    ROOT_FILE_MC_MUON = ROOT.TFile('muons.root', 'OPEN')
    HISTO_MC_MUON = ROOT_FILE_MC_MUON.Get(var_name)
    HISTO_MC_MUON.SetLineColor(3)
    HISTO_MC_MUON.SetMaximum(1000)

    if var_name.find('hadron')!=0:
        HISTO_MC_MUON.Scale(HISTO_INPUT.Integral()/HISTO_MC_MUON.Integral())
    
    if var_name.find('muon')!=0:
        HISTO_MC_HADRON.Scale(HISTO_INPUT.Integral()/HISTO_MC_HADRON.Integral())


    # Plot the three histogram in the same canvas
    HISTO_INPUT.Draw()
    HISTO_MC_HADRON.Draw("same")
    HISTO_MC_MUON.Draw("same")
    
    # Set the legends
    # TLegend(x1, y1, x2, y2, "header", "option")
    legend = ROOT.TLegend(0.15, 0.70, 0.40, 0.85)
    LEGEND_INPUT = data_name.replace('.root','')
    legend.AddEntry(HISTO_INPUT, LEGEND_INPUT, 'l')
    legend.AddEntry(HISTO_MC_HADRON,'MC of Hadrons','l')
    legend.AddEntry(HISTO_MC_MUON,'MC of Muons','l')
    legend.Draw("same")
    
    # Draw and save
    canvas1.Draw()
    canvas1.Print(LEGEND_INPUT + '_' + var_name + '.pdf')

In [14]:
# [event_var]
# 'n_elec','n_muon','n_hadron','n_total','max_pt','E_total','pass_hadron_selection','pass_muon_selection'
# with [cut_names]
# 'hadron_cut','muon_cut'

# Hadrons
compare_with_mc('91gev.root','hadron_cut_E_total')
# Muons
compare_with_mc('91gev.root','muon_cut_E_total')

#if you want to open a pdf file in python notebooks just do
#IFrame('91gev_hadron_cut_E_total.pdf', width=600, height=400)
IFrame('91gev_muon_cut_E_total.pdf', width=600, height=400)

## Hadron Fit

In [15]:
# Definition of stuff you don't really need to look into
# Here we define the functions for the convolution between the Breit-Wigner and the gaussian
def radcorr(z,s):
    me2    = 0.0000002611200
    alphpi = 0.0023228196
    zeta2  = 1.644934
    cons1  = -2.16487
    cons2  = 9.8406
    cons3  = -6.26
    fac2   = 1.083
    l      = np.log(s/me2)
    beta   = 2.*alphpi*(l-1.)
    del1vs = alphpi*((3./2.)*l+2.*zeta2-2.)
    del2vs = pow(alphpi,2.)*(cons1*pow(l,2.)+cons2*l+cons3)
    delvs  = 1.+del1vs+del2vs
    del1h  = -alphpi*(1.+z)*(l-1.)
    delbh  = pow(alphpi,2.)*(1./2.)*pow((l-1.),2.)*((1.+z)*(3.*np.log(z)-4.*np.log(1.-z))-(4./(1.-z))*np.log(z)-5.-z)
    rad    = beta*pow((1.-z),beta-1.)*delvs+del1h+delbh
    rad *=fac2
    return rad

def bw(s, s0, g2, m2):
    return s0*(s*g2)/(pow(s-m2,2)+m2*g2)

def bw2(x,pars):
    return bw(pow(x[0],2),pars[0],pow(pars[1],2),pow(pars[2],2))

def integrand(x,pars):
    return bw(x[0]*pars[0],pars[1],pars[2],pars[3])*radcorr(x[0],pars[0])

integrandfunc=ROOT.TF1("integrandfunc",integrand,0,1,4)

def convolution(s,s0,g2,m2): 
    if(s<60):
        print("s too small")
        return -1

    zmin = pow(60,2)/s
    zmax = 0.9999999999
    integrandfunc.SetParameters(s,s0,g2,m2)
    result = integrandfunc.Integral(zmin,zmax,0.000001);
    return result;

def convolution2(x,pars):
    return convolution(pow(x[0],2), pars[0], pow(pars[1],2), pow(pars[2],2))

In [16]:
def efficiency(data_name):
    """ Calculate the efficiency (and its uncertainty)
    of the selection from the given Monte-Carlo ROOT-file. """
    
    ROOT_FILE_MC = ROOT.TFile(data_name, 'OPEN')
    
    if data_name.find('hadron')!= -1:
        histo_name = 'hadron_cut_pass_hadron_selection'
    elif data_name.find('muon')!= -1:
        histo_name = 'muon_cut_pass_muon_selection'
    else:
        raise ValueError('The efficiency should only be determined by Monte-Carlo data.')
    
    HISTO_PASS_SELECTION = ROOT_FILE_MC.Get(histo_name)
    HISTO_TOTAL = ROOT_FILE_MC.Get('n_total')
    
    N_pass = HISTO_PASS_SELECTION.GetEntries()
    N_total = HISTO_TOTAL.GetEntries()
    eps = N_pass / N_total
    err_eps = np.sqrt(N_pass) / N_total # Poisson distribution
    
    ROOT_FILE_MC.Close()
    
    return eps, err_eps

In [17]:
def cross_section(n_cut, err_n_cut, n_bkg, eps, err_eps, lum, err_lum):
    """ Evaluate the cross section with its uncertainty. """
    
    XS = (n_cut - n_bkg) / (eps * lum)
    err_XS = np.zeros(3, dtype='float')
    
    for i in np.arange(3):
        err_XS[i] = np.sqrt(np.sum(np.square([err_n_cut[i]/(eps*lum[i]), 
                                              -(n_cut[i]-n_bkg)/(eps**2*lum[i])*err_eps, 
                                              -(n_cut[i]-n_bkg)/(eps*lum[i]**2)*err_lum[i]])))
    
    return XS, err_XS

In [18]:
# Number of events which pass the selection
N_had = np.zeros(3, dtype='int')
N_bkg = 1
root_real_data = ['89gev.root', '91gev.root', '93gev.root']

for i, file_name in enumerate(root_real_data):
    ROOT_FILE = ROOT.TFile(file_name, 'OPEN')
    HISTO = ROOT_FILE.Get('hadron_cut_pass_hadron_selection')
    N_had[i] = HISTO.GetEntries()
    ROOT_FILE.Close()

err_N_had = np.around(np.sqrt(N_had))
print('# Hadronic Events:',N_had,'+/-',err_N_had)

# Efficiency of hadronic selection
eps_had, err_eps_had = efficiency('hadrons.root')
print('# Hadron Cut Efficiency:',eps_had,'+/-',err_eps_had)

# Define the integrated luminosity and its uncertainty
L = np.asarray([179.3, 135.9, 151.1]) # in nb^-1
err_L = 0.01 * L

# Calculation of the hadronic cross section
XS_had, err_XS_had = cross_section(N_had, err_N_had, N_bkg, eps_had, err_eps_had, L, err_L)

print(XS_had)
print(err_XS_had)

# Hadronic Events: [1802 4014 2143] +/- [42. 63. 46.]
# Hadron Cut Efficiency: 0.9931 +/- 0.009965440281292141
[10.11440737 29.73423169 14.27453666]
[0.27598339 0.62875964 0.36724181]


In [19]:
# Perform a fit for hadrons
# Decleare the energy, the cross section and their errors
npoints=3
x = [89.48, 91.33, 93.03]
y = XS_had
ex = [0, 0, 0]
ey = err_XS_had

# We define the function for the fit
# TF1(name, function, xmin, xmax, npar)
convolutionfunc = ROOT.TF1("convolutionfunc",convolution2,80,100,3) # Do not decrease the first term
convolutionfunc.SetNpx(200)
convolutionfunc.SetLineColor(8)
convolutionfunc.SetParNames("sigma","gamma","mass")

# We define a TGraph on which we will perform the fit
graph = ROOT.TGraphErrors(npoints,np.array(x),np.array(y),np.array(ex),np.array(ey))
graph2 = ROOT.TGraph(npoints,np.array(x),np.array(y))

# We initialize the paramters
convolutionfunc.SetParameters(40,2,90)
ROOT.gROOT.ProcessLine("gErrorIgnoreLevel = 9000;")

# And then we conduct the fit
fitres = graph.Fit(convolutionfunc,"EX0 V","",70,150)
print(type(fitres))
sigma_had = convolutionfunc.GetParameter(0)
gamma_had = abs(convolutionfunc.GetParameter(1))
mass_had = abs(convolutionfunc.GetParameter(2))
error_sigma_had = convolutionfunc.GetParError(0)
error_gamma_had = abs(convolutionfunc.GetParError(1))
error_mass_had = abs(convolutionfunc.GetParError(2))
chi2_had = convolutionfunc.GetChisquare()

# Printing the results
print("sigma",convolutionfunc.GetParameter(0),"pm",convolutionfunc.GetParError(0))
print("gamma",abs(convolutionfunc.GetParameter(1)),"pm",convolutionfunc.GetParError(1))
print("mass",abs(convolutionfunc.GetParameter(2)),"pm",convolutionfunc.GetParError(2))
print("chi^2",convolutionfunc.GetChisquare())

<class 'ROOT.TFitResultPtr'>
sigma 37.060040510578176 pm 0.8606551326956331
gamma 2.5904006850352737 pm 0.0687342580292568
mass 91.19871360352445 pm 0.03101078558285274
chi^2 1.7184458175853625e-07
 **********
 **    7 **SET PRINT           2
 **********
 **********
 **    8 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 sigma        4.00000e+01  1.20000e+01     no limits
     2 gamma        2.00000e+00  6.00000e-01     no limits
     3 mass         9.00000e+01  2.70000e+01     no limits
 **********
 **    9 **SET ERR           1
 **********
 **********
 **   10 **SET PRINT           2
 **********
 **********
 **   11 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   12 **MIGRAD        1345        0.01
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-05
 FC

We want to compare the Breit Wigner function without QED corrections with the fit function in a same graph for hadron.

In [20]:
# Create a new canvas for display
canvas = ROOT.TCanvas("canvas","Fit Result")

# Plot the Breit Wigner function without QED corrections
wigner = ROOT.TF1("Breit Wigner Fit",bw2,80,100,3)
wigner.SetNpx(200)
wigner.SetParameters(convolutionfunc.GetParameter(0),abs(convolutionfunc.GetParameter(1)),abs(convolutionfunc.GetParameter(2)))
wigner.SetLineColor(2)
wigner.GetXaxis().SetTitle("Center-of-Mass Energy (GeV)")
wigner.GetYaxis().SetTitle("Cross Section (nb)")
wigner.Draw()

# Plot the fit function and data points
convolutionfunc.Draw("same")
graph.Draw("*same")

# Set the legends
# ROOT.gStyle.SetOptFit(00000000)
legend = ROOT.TLegend(0.15, 0.70, 0.40, 0.85)
legend.AddEntry(wigner, "Breit-Wigner", "l")
legend.AddEntry(graph, "Data points", "lep")
legend.AddEntry(convolutionfunc, "QED corrected", "l")
legend.Draw("same")

canvas.Print('bw_qed_hadrons.pdf')
IFrame('bw_qed_hadrons.pdf', width=600, height=400)

## Muon Fit

In [21]:
# Number of events which pass the selection
N_muon = np.zeros(3, dtype='int')
N_bkg = 4
root_real_data = ['89gev.root', '91gev.root', '93gev.root']

for i, file_name in enumerate(root_real_data):
    ROOT_FILE = ROOT.TFile(file_name, 'OPEN')
    HISTO = ROOT_FILE.Get('muon_cut_pass_muon_selection')
    N_muon[i] = HISTO.GetEntries()
    ROOT_FILE.Close()

err_N_muon = np.around(np.sqrt(N_muon))
print('# Muonic Events:',N_muon,'+/-',err_N_muon)

# Efficiency of hadronic selection
eps_muon, err_eps_muon = efficiency('muons.root')
print('# Muon Cut Efficiency:',eps_muon,'+/-',err_eps_muon)

# Define the integrated luminosity and its uncertainty
L = np.asarray([179.3, 135.9, 151.1]) # in nb^-1
err_L = 0.01 * L

# Calculation of the hadronic cross section
XS_muon, err_XS_muon = cross_section(N_muon, err_N_muon, N_bkg, eps_muon, err_eps_muon, L, err_L)

print(XS_muon)
print(err_XS_muon)

# Muonic Events: [ 65 113  67] +/- [ 8. 11.  8.]
# Muon Cut Efficiency: 0.4626341659143344 +/- 0.006812288833925026
[0.73538005 1.73368159 0.90123569]
[0.0973275  0.17765926 0.11556145]


Now we want to perform a fit for muons, try implement it by yourself.

In [22]:
# Perform a fit for muons
# Decleare the energy, the cross section and their errors
npoints=3
x = [89.48, 91.33, 93.03]
y = XS_muon
ex = [0, 0, 0]
ey = err_XS_muon

# We define the function for the fit
# TF1(name, function, xmin, xmax, npar)
convolutionfunc = ROOT.TF1("convolutionfunc",convolution2,80,100,3) # Do not decrease the first term
convolutionfunc.SetNpx(200)
convolutionfunc.SetLineColor(8)
convolutionfunc.SetParNames("sigma","gamma","mass")

# We define a TGraph on which we will perform the fit
graph = ROOT.TGraphErrors(npoints,np.array(x),np.array(y),np.array(ex),np.array(ey))
graph2 = ROOT.TGraph(npoints,np.array(x),np.array(y))

# We initialize the paramters
convolutionfunc.SetParameters(2,2,90)
ROOT.gROOT.ProcessLine("gErrorIgnoreLevel = 9000;")

# And then we conduct the fit
fitres = graph.Fit(convolutionfunc,"EX0 V","",70,150)

# Printing the results
print("Sigma",convolutionfunc.GetParameter(0),"pm",convolutionfunc.GetParError(0))
print("Gamma",abs(convolutionfunc.GetParameter(1)),"pm",convolutionfunc.GetParError(1))
print("Mass",abs(convolutionfunc.GetParameter(2)),"pm",convolutionfunc.GetParError(2))

sigma_muon = convolutionfunc.GetParameter(0)
gamma_muon = abs(convolutionfunc.GetParameter(1))
mass_muon = abs(convolutionfunc.GetParameter(2))
error_sigma_muon = convolutionfunc.GetParError(0)
error_gamma_muon = abs(convolutionfunc.GetParError(1))
error_mass_muon = abs(convolutionfunc.GetParError(2))
chi2_muon = convolutionfunc.GetChisquare()

print("chi^2",chi2_muon)

Sigma 2.1359722859966124 pm 0.24964298613819416
Gamma 2.960942231433993 pm 0.43696390595789925
Mass 91.1034420435971 pm 0.1668796423666415
chi^2 5.762752163311849e-11
 **********
 **   13 **SET PRINT           2
 **********
 **********
 **   14 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 sigma        2.00000e+00  6.00000e-01     no limits
     2 gamma        2.00000e+00  6.00000e-01     no limits
     3 mass         9.00000e+01  2.70000e+01     no limits
 **********
 **   15 **SET ERR           1
 **********
 **********
 **   16 **SET PRINT           2
 **********
 **********
 **   17 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   18 **MIGRAD        1345        0.01
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-05
 FCN=72.3042 FROM MIGRAD    STATUS

In [24]:
# Perform a fit for muon with fixed Z boson mass and width 
# Decleare the energy, the cross section and their errors
npoints=3
x = [89.48, 91.33, 93.03]
y = XS_muon
ex = [0, 0, 0]
ey = err_XS_muon

# We define the function for the fit
# TF1(name, function, xmin, xmax, npar)
convolutionfunc = ROOT.TF1("convolutionfunc",convolution2,80,100,3) # Do not decrease the first term
convolutionfunc.SetNpx(200)
convolutionfunc.SetLineColor(8)
convolutionfunc.SetParNames("sigma","gamma","mass")

# We define a TGraph on which we will perform the fit
graph = ROOT.TGraphErrors(npoints,np.array(x),np.array(y),np.array(ex),np.array(ey))
graph2 = ROOT.TGraph(npoints,np.array(x),np.array(y))

# We initialize and fix the paramters
convolutionfunc.SetParameters(2,2,90)
convolutionfunc.FixParameter(1, 2.5904)
convolutionfunc.FixParameter(2, 91.1987)
ROOT.gROOT.ProcessLine("gErrorIgnoreLevel = 9000;")

# And then we conduct the fit
fitres = graph.Fit(convolutionfunc,"EX0 V","",70,150)

# Printing the results
print("Sigma",convolutionfunc.GetParameter(0),"pm",convolutionfunc.GetParError(0))

sigma_muon_fix = convolutionfunc.GetParameter(0)
error_sigma_muon_fix = convolutionfunc.GetParError(0)
chi2_muon = convolutionfunc.GetChisquare()

print("chi^2",chi2_muon)

Sigma 2.3179163226058783 pm 0.1594415196984987
chi^2 1.6237817848203304
 **********
 **   27 **SET PRINT           2
 **********
 **********
 **   28 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 sigma        2.00000e+00  6.00000e-01     no limits
     2 gamma        2.59040e+00  2.59040e-01     no limits
 **********
 **   29 **FIX           2
 **********
 FCN= unknown        FROM FIX         STATUS=RESET           0 CALLS           0 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS      PHYSICAL LIMITS       
  NO.   NAME      VALUE            ERROR       NEGATIVE      POSITIVE  
   1  sigma        2.00000e+00   6.00000e-01
   2  gamma        2.59040e+00     fixed    
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     3 mass         9.11987e+01  9.11987e+00     no limits
 **********
 **   30 **FIX           

We also want to see how the breit-wigner would look for muon.

In [25]:
# Create a new canvas for display
canvas = ROOT.TCanvas("canvas","Fit Result")

# Plot the Breit Wigner function without QED corrections
wigner = ROOT.TF1("Breit Wigner Fit",bw2,80,100,3)
wigner.SetNpx(200)
wigner.SetParameters(convolutionfunc.GetParameter(0),abs(convolutionfunc.GetParameter(1)),abs(convolutionfunc.GetParameter(2)))
wigner.SetLineColor(2)
wigner.GetXaxis().SetTitle("Center-of-Mass Energy (GeV)")
wigner.GetYaxis().SetTitle("Cross Section (nb)")
wigner.Draw()

# Plot the fit function and data points
convolutionfunc.Draw("same")
graph.Draw("*same")

# Set the legends
# ROOT.gStyle.SetOptFit(00000000)
legend = ROOT.TLegend(0.15, 0.70, 0.40, 0.85)
legend.AddEntry(wigner, "Breit-Wigner", "l")
legend.AddEntry(graph, "Data points", "lep")
legend.AddEntry(convolutionfunc, "QED corrected", "l")
legend.Draw("same")

canvas.Print('bw_qed_muons.pdf')
IFrame('bw_qed_muons.pdf', width=600, height=400)

Compute the partial width of the electrons using the muon XS.

In [26]:
def partial_width_mu(mass_z, err_mass_z, gamma_z, err_gamma_z, sigma_peak, err_sigma_peak):
    
    # Define three symbols for mass, gamma and sigma
    M, G, S = sympy.symbols('M G S', real=True)
    f = sympy.sqrt((M**2 * G**2 * S * 2.568e-6)/ (12 * sympy.pi))
    gamma_mu = float(sympy.N(f.subs({M:mass_z, G:gamma_z, S:sigma_peak})))
    
    dfdM = sympy.diff(f, M)
    dfdM = float(sympy.N(dfdM.subs({M:mass_z, G:gamma_z, S:sigma_peak})))
    dfdG = sympy.diff(f, G)
    dfdG = float(sympy.N(dfdG.subs({M:mass_z, G:gamma_z, S:sigma_peak})))
    dfdS = sympy.diff(f, S)
    dfdS = float(sympy.N(dfdS.subs({M:mass_z, G:gamma_z, S:sigma_peak})))
    
    df = np.asarray([dfdM, dfdG, dfdS])
    err = np.asarray([err_mass_z, err_gamma_z, err_sigma_peak])
    err_gamma_mu = np.sqrt(np.sum(np.square([df*err])))
    
    return gamma_mu, err_gamma_mu

# Execution
ga_mu, err_ga_mu = partial_width_mu(91.20, 0.03, 2.59, 0.07, 2.31, 0.16)
print(ga_mu)
print(err_ga_mu)

0.09369847934654682
0.0041162822481817535


The second way to determine the partial width. 

Compute the partial width of the neutrino and then the partial width of the electrons using the hadronic XS.

In [27]:
def partial_width_nu(mass_z, err_mass_z):
    
    M = sympy.symbols('M', real=True)
    GF = 1.16638e-5 # GeV^(-2)
    f = (GF * M**3) / (12 * sympy.sqrt(2)* sympy.pi)
    
    dfdM = sympy.diff(f, M)
    dfdM = float(sympy.N(dfdM.subs(M,mass_z)))
    
    gamma_nu = float(sympy.N(f.subs(M,mass_z)))
    err_gamma_nu = np.sqrt(np.square(dfdM * err_mass_z))
    
    return gamma_nu, err_gamma_nu

# Execution
ga_nu, err_ga_nu = partial_width_nu(91.20, 0.03)
print(ga_nu)
print(err_ga_nu)

0.16595046076384357
0.00016376690206958245


In [29]:
def partial_width_el(mass_z, err_mass_z, gamma_z, err_gamma_z, gamma_nu, err_gamma_nu, sigma_peak, err_sigma_peak):
    
    # Define four symbols for Z mass, Z width, neutrino width and sigma
    M, Gz, Gn, S = sympy.symbols('M Gz Gn S', real=True)
    A = 3
    B = 3 * Gn - Gz
    C = (M**2 * Gz**2 * S * 2.568e-6)/ (12.* sympy.pi)
    f = (-B - sympy.sqrt(B**2 - 4 * A * C)) / (2 * A)
    gamma_el = float(sympy.N(f.subs({M:mass_z, Gz:gamma_z, Gn:gamma_nu, S:sigma_peak})))
    
    dfdM = sympy.diff(f, M)
    dfdM = float(sympy.N(dfdM.subs({M:mass_z, Gz:gamma_z, Gn:gamma_nu, S:sigma_peak})))
    dfdGz = sympy.diff(f, Gz)
    dfdGz = float(sympy.N(dfdGz.subs({M:mass_z, Gz:gamma_z, Gn:gamma_nu, S:sigma_peak})))
    dfdGn = sympy.diff(f, Gn)
    dfdGn = float(sympy.N(dfdGn.subs({M:mass_z, Gz:gamma_z, Gn:gamma_nu, S:sigma_peak})))
    dfdS = sympy.diff(f, S)
    dfdS = float(sympy.N(dfdS.subs({M:mass_z, Gz:gamma_z, Gn:gamma_nu, S:sigma_peak})))
    
    df = np.asarray([dfdM, dfdGz, dfdGn, dfdS])
    err = np.asarray([err_mass_z, err_gamma_z, err_gamma_nu, err_sigma_peak])
    err_gamma_el = np.sqrt(np.sum(np.square([df*err])))
    
    return gamma_el, err_gamma_el

# Execution
ga_el, err_ga_el = partial_width_el(91.20, 0.03, 2.59, 0.07, ga_nu, err_ga_nu, 37.1, 0.9)
print(ga_el)
print(err_ga_el)

0.07558914314667001
0.002526662474259997


Compute the Weinberg angle.

In [30]:
def weinberg_angle(mass_z, err_mass_z, gamma_el, err_gamma_el):
    
    M, Ge = sympy.symbols('M Ge',real=True)
    GF = 1.16638e-5 # GeV^(-2)
    f = (1 - sympy.sqrt((24 * sympy.sqrt(2) * sympy.pi * Ge) / (GF * M**3) - 1)) / 4
    theta = sympy.Abs(sympy.N(f.subs({M:mass_z, Ge:gamma_el})))
    
    dfdM = sympy.diff(f, M)
    dfdM = sympy.Abs(sympy.N(dfdM.subs({M:mass_z, Ge:gamma_el})))
    dfdGe = sympy.diff(f, Ge)
    dfdGe = sympy.Abs(sympy.N(dfdGe.subs({M:mass_z, Ge:gamma_el})))
    
    df = np.asarray([float(dfdM), float(dfdGe)])
    err = np.asarray([err_mass_z, err_gamma_el])
    err_theta = np.sqrt(np.sum(np.square([df*err])))
    
    return float(theta), float(err_theta)

# Execution
sin2th, err_sin2th = weinberg_angle(91.20, 0.03, ga_el, err_ga_el)
print(sin2th)
print(err_sin2th)

0.2608897717036216
0.01276335878616843


Compute the color parameter.

In [35]:
# Hadronic partial width
ga_z = 2.59
err_ga_z = 0.07
ga_had = ga_z - 3*ga_nu - 3*ga_el
err_ga_had = np.sqrt(np.sum(np.square([err_ga_z, err_ga_nu, err_ga_el])))
print(ga_had)
print(err_ga_had)

# Or
"""
def partial_width_had(mass_z, err_mass_z, gamma_z, err_gamma_z, gamma_el, err_gamma_el, sigma_peak, err_sigma_peak):
    
    # Define four symbols for Z mass, Z width, electron width and sigma
    M, Gz, Ge, S = sympy.symbols('M Gz Gn S', real=True)
    f = (M**2 * Gz**2 * S * 2.568e-6)/ (12.* sympy.pi * Ge)
    gamma_had = float(sympy.N(f.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))
    
    dfdM = sympy.diff(f, M)
    dfdM = float(sympy.N(dfdM.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))
    dfdGz = sympy.diff(f, Gz)
    dfdGz = float(sympy.N(dfdGz.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))
    dfdGe = sympy.diff(f, Ge)
    dfdGe = float(sympy.N(dfdGe.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))
    dfdS = sympy.diff(f, S)
    dfdS = float(sympy.N(dfdS.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))
    
    df = np.asarray([dfdM, dfdGz, dfdGe, dfdS])
    err = np.asarray([err_mass_z, err_gamma_z, err_gamma_el, err_sigma_peak])
    err_gamma_had = np.sqrt(np.sum(np.square([df*err])))
    
    return gamma_had, err_gamma_had
    
"""

1.8653811882684592
0.07004577676674767


"\ndef partial_width_had(mass_z, err_mass_z, gamma_z, err_gamma_z, gamma_el, err_gamma_el, sigma_peak, err_sigma_peak):\n    \n    # Define four symbols for Z mass, Z width, electron width and sigma\n    M, Gz, Ge, S = sympy.symbols('M Gz Gn S', real=True)\n    f = (M**2 * Gz**2 * S * 2.568e-6)/ (12.* sympy.pi * Ge)\n    gamma_had = float(sympy.N(f.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))\n    \n    dfdM = sympy.diff(f, M)\n    dfdM = float(sympy.N(dfdM.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))\n    dfdGz = sympy.diff(f, Gz)\n    dfdGz = float(sympy.N(dfdGz.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))\n    dfdGe = sympy.diff(f, Ge)\n    dfdGe = float(sympy.N(dfdGe.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))\n    dfdS = sympy.diff(f, S)\n    dfdS = float(sympy.N(dfdS.subs({M:mass_z, Gz:gamma_z, Ge:gamma_el, S:sigma_peak})))\n    \n    df = np.asarray([dfdM, dfdGz, dfdGe, dfdS])\n    err = np.asarray([err_mass_z, err_gamma_z, er

In [36]:
def partial_width_quark(mass_z, err_mass_z, sin2_thw, err_sin2_thw):
    
    # Define four symbols for Z mass, Z width, neutrino width and sigma
    M, SIN = sympy.symbols('M S', real=True)
    GF = 1.16638e-5 # GeV^(-2)
    Qf = 1/3
    f = (GF * M**3) / (24 * sympy.sqrt(2)* sympy.pi) * (1 + (1 - 4 * Qf * SIN)**2)
    gamma_quark = float(sympy.N(f.subs({M:mass_z, SIN:sin2_thw})))
    
    dfdM = sympy.diff(f, M)
    dfdM = float(sympy.N(dfdM.subs({M:mass_z, SIN:sin2_thw})))
    dfdSIN = sympy.diff(f, SIN)
    dfdSIN = float(sympy.N(dfdSIN.subs({M:mass_z, SIN:sin2_thw})))
    
    df = np.asarray([dfdM, dfdSIN])
    err = np.asarray([err_mass_z, err_sin2_thw])
    err_gamma_quark = np.sqrt(np.sum(np.square([df*err])))
    
    return gamma_quark, err_gamma_quark

# Execution
ga_qua, err_ga_qua = partial_width_quark(91.20, 0.03, sin2th, err_sin2th)
print(ga_qua)
print(err_ga_qua)

0.11826423673536515
0.0018454312972210958


In [37]:
def num_color(gamma_had, err_gamma_had, gamma_u, err_gamma_u, gamma_d, err_gamma_d):
    
    Gh, Gu, Gd = sympy.symbols('Gh Gu Gd', real=True)
    f = Gh / (1.04 * (2 * Gu + 3 * Gd))
    color = float(sympy.N(f.subs({Gh:gamma_had, Gu:gamma_u, Gd:gamma_d})))
    
    dfdGh = sympy.diff(f, Gh)
    dfdGh = sympy.N(dfdGh.subs({Gh:gamma_had, Gu:gamma_u, Gd:gamma_d}))
    dfdGu = sympy.diff(f, Gu)
    dfdGu = sympy.N(dfdGu.subs({Gh:gamma_had, Gu:gamma_u, Gd:gamma_d}))
    dfdGd = sympy.diff(f, Gd)
    dfdGd = sympy.N(dfdGd.subs({Gh:gamma_had, Gu:gamma_u, Gd:gamma_d}))
    
    df = np.asarray([float(dfdGh), float(dfdGu), float(dfdGd)])
    err = np.asarray([err_gamma_had, err_gamma_u, err_gamma_d])
    err_color = np.sqrt(np.sum(np.square([df*err])))
    
    return color, err_color

# Execution
N_C, err_N_C = num_color(ga_had, err_ga_had, 0.0907, 0.0017, 0.1183, 0.0018)
print(N_C)
print(err_N_C)

3.344463468115684
0.13173994170779602
